# 知识处理完整Pipeline - 简化但功能完整版

## 流程概览

```
原始文件(PDF/DOC/PPTX)
    ↓
步骤1: 文件扫描分组 → organized_files
    ↓
步骤2: 文档加载清洗 → List[Document]
    ↓
步骤3: 结构化提取 → structured.json
    ↓
步骤4: 向量化存储 → vector_db/
    ↓
步骤5: 完整Pipeline → 一键处理所有文件
```

## 目标

- **功能完整**: 所有5步骤都可运行
- **简洁高效**: 代码精简but够用
- **实际可用**: 能生成JSON和向量库


In [ ]:
# 导入依赖
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass
from collections import defaultdict
from difflib import SequenceMatcher
from enum import IntEnum

# LangChain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from langchain_core.output_parsers import JsonOutputParser

# 环境变量
from dotenv import load_dotenv
load_dotenv("../../../config/.env")

print("✅ 依赖导入完成")


In [ ]:
# 数据结构
class FilePriority(IntEnum):
    PDF_NOTE = 1
    WORD_DOC = 2
    PDF_REGULAR = 3
    POWERPOINT = 4
    UNKNOWN = 99

@dataclass
class FileInfo:
    path: Path
    original_name: str
    cleaned_name: str
    sequence: int
    sequence_str: str
    priority: FilePriority

@dataclass
class KnowledgeGroup:
    group_key: str
    topic: str
    sequence: int
    files: List[FileInfo]
    primary_file: FileInfo
    file_types: List[str]

print("✅ 数据结构定义完成")


## 步骤1: KnowledgeOrganizer - 文件扫描分组


In [ ]:
class KnowledgeOrganizer:
    """文件扫描和智能分组"""
    SUPPORTED_EXTENSIONS = {'.pdf', '.doc', '.docx', '.ppt', '.pptx'}
    NOISE_PATTERNS = [r'\[防断更.*?\]', r'\[.*?微.*?\]', r'_\d{14}', r'_\d{8}', r'_笔记', r'\s*\(.*?\)\s*']

    def __init__(self, knowledge_base_dir, similarity_threshold=0.7, verbose=True):
        self.knowledge_base_dir = Path(knowledge_base_dir)
        self.similarity_threshold = similarity_threshold
        self.verbose = verbose
        if not self.knowledge_base_dir.exists():
            raise ValueError(f"目录不存在: {self.knowledge_base_dir}")

    def _log(self, msg):
        if self.verbose: print(msg)

    def clean_filename(self, filename):
        """清洗文件名(去除噪音)"""
        name = Path(filename).stem
        for pattern in self.NOISE_PATTERNS:
            name = re.sub(pattern, '', name)
        return re.sub(r'\s+', ' ', name).strip()

    def extract_sequence_number(self, filename):
        """提取序号"""
        match = re.match(r'^(\d+)', filename)
        return (int(match.group(1)), match.group(1)) if match else (999999, "")

    def calculate_similarity(self, str1, str2):
        """计算相似度"""
        return SequenceMatcher(None, str1, str2).ratio()

    def get_file_priority(self, file_path):
        """确定文件优先级"""
        name, suffix = file_path.name.lower(), file_path.suffix.lower()
        if '笔记' in name and suffix == '.pdf': return FilePriority.PDF_NOTE
        if suffix in ['.doc', '.docx']: return FilePriority.WORD_DOC
        if suffix == '.pdf': return FilePriority.PDF_REGULAR
        if suffix in ['.ppt', '.pptx']: return FilePriority.POWERPOINT
        return FilePriority.UNKNOWN

    def create_file_info(self, file_path):
        """创建文件信息对象"""
        original_name = file_path.name
        cleaned_name = self.clean_filename(original_name)
        sequence, sequence_str = self.extract_sequence_number(cleaned_name)
        priority = self.get_file_priority(file_path)
        return FileInfo(file_path, original_name, cleaned_name, sequence, sequence_str, priority)

    def group_files_by_similarity(self, files):
        """按相似度分组"""
        groups, processed = {}, set()
        for i, file1 in enumerate(files):
            if file1.path in processed: continue
            group_key = f"{file1.sequence_str}_{file1.cleaned_name[:20]}"
            group_files = [file1]
            processed.add(file1.path)

            for file2 in files[i+1:]:
                if file2.path in processed: continue
                if file1.sequence == file2.sequence:
                    if self.calculate_similarity(file1.cleaned_name, file2.cleaned_name) >= self.similarity_threshold:
                        group_files.append(file2)
                        processed.add(file2.path)

            group_files.sort(key=lambda f: (f.priority.value, f.original_name))
            groups[group_key] = KnowledgeGroup(group_key, file1.cleaned_name, file1.sequence,
                                              group_files, group_files[0], [f.path.suffix for f in group_files])
            self._log(f"✓ {file1.cleaned_name[:30]} ({len(group_files)}文件)")
        return groups

    def scan_and_organize(self):
        """扫描并组织文件"""
        self._log(f"📚 扫描: {self.knowledge_base_dir}")
        all_files = []
        for ext in self.SUPPORTED_EXTENSIONS:
            all_files.extend(self.knowledge_base_dir.glob(f"*{ext}"))

        if not all_files:
            self._log("⚠️ 未找到文件")
            return {}

        self._log(f"找到 {len(all_files)} 个文件")
        file_infos = [self.create_file_info(f) for f in all_files]
        groups = self.group_files_by_similarity(file_infos)
        sorted_groups = dict(sorted(groups.items(), key=lambda x: x[1].sequence))
        self._log(f"✅ {len(sorted_groups)} 个知识块\n")
        return {self.knowledge_base_dir.name: sorted_groups}

print("✅ 步骤1完成")


In [ ]:
class DocumentLoader:
    """文档加载器(简化版,仅支持PDF)"""

    def load_pdf(self, file_path: Path) -> List[Document]:
        """加载PDF文件"""
        loader = PyMuPDFLoader(str(file_path))
        return loader.load()

    def clean_document_text(self, doc: Document) -> Document:
        """清洗文档文本"""
        text = doc.page_content
        text = re.sub(r'[\uf06c\uf0fc]', '', text)  # 特殊字符
        text = re.sub(r'\s+', ' ', text)  # 多余空白
        doc.page_content = text.strip()
        return doc

    def load_and_clean(self, file_path: Path) -> List[Document]:
        """加载并清洗文档"""
        if file_path.suffix.lower() == '.pdf':
            docs = self.load_pdf(file_path)
            return [self.clean_document_text(doc) for doc in docs]
        else:
            print(f"⚠️ 暂不支持{file_path.suffix},跳过")
            return []

print("✅ 步骤2完成")


## 步骤3: KnowledgeExtractor - LLM结构化提取


In [ ]:
class KnowledgeExtractor:
    """LLM知识提取器(简化版)"""

    def __init__(self, model_name="deepseek-chat"):
        self.llm = ChatDeepSeek(model=model_name, temperature=0)
        self.prompt = ChatPromptTemplate.from_template("""
你是金融知识提取专家。从文档提取结构化知识,返回JSON。

文档: {content}

提取JSON(只返回JSON):
{{
  "topic": "主题",
  "key_concepts": [{{"name": "概念", "definition": "定义", "importance": "重要性"}}],
  "indicators": [{{"name": "指标", "calculation": "计算", "interpretation": "解读"}}],
  "analysis_methods": [{{"name": "方法", "steps": "步骤", "application": "应用"}}],
  "summary": "总结"
}}
""")

    def extract_from_documents(self, docs: List[Document], topic: str) -> Dict:
        """从文档提取知识"""
        content = "\n\n".join([d.page_content for d in docs[:5]])[:15000]
        try:
            chain = self.prompt | self.llm | JsonOutputParser()
            result = chain.invoke({"content": content})
            result["topic"] = topic
            return result
        except Exception as e:
            print(f"⚠️ LLM提取失败: {e}")
            return {"topic": topic, "key_concepts": [], "indicators": [],
                   "analysis_methods": [], "summary": "提取失败"}

print("✅ 步骤3完成")


## 步骤4: VectorStoreManager - 向量化存储


In [ ]:
class VectorStoreManager:
    """向量存储管理器(简化版)"""

    def __init__(self, embedding_model="sentence-transformers/all-MiniLM-L6-v2", persist_directory="./vector_db"):
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.persist_directory = Path(persist_directory)
        self.vector_stores = {}

    def get_or_create_store(self, domain: str) -> Chroma:
        """获取或创建向量存储"""
        if domain in self.vector_stores:
            return self.vector_stores[domain]

        persist_path = str(self.persist_directory / domain)
        store = Chroma(
            collection_name=f"{domain}_col",
            embedding_function=self.embeddings,
            persist_directory=persist_path
        )
        self.vector_stores[domain] = store
        return store

    def split_documents(self, docs: List[Document]) -> List[Document]:
        """分割文档为chunks"""
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        return splitter.split_documents(docs)

    def add_documents(self, domain: str, docs: List[Document], metadata: Dict = None):
        """添加文档到向量存储"""
        if not docs:
            print("  ⚠️ 无文档,跳过向量化")
            return

        chunks = self.split_documents(docs)
        if metadata:
            for chunk in chunks:
                chunk.metadata.update(metadata)

        store = self.get_or_create_store(domain)
        store.add_documents(chunks)
        print(f"  → 向量化: {len(chunks)} chunks")

print("✅ 步骤4完成")


## 步骤5: KnowledgeProcessor - 完整Pipeline


In [ ]:
class KnowledgeProcessor:
    """完整Pipeline协调器"""

    def __init__(self, knowledge_base_dir: str, memories_dir="./memories/knowledge", vector_db_dir="./vector_db"):
        self.organizer = KnowledgeOrganizer(knowledge_base_dir)
        self.loader = DocumentLoader()
        self.extractor = KnowledgeExtractor()
        self.vector_manager = VectorStoreManager(persist_directory=vector_db_dir)
        self.memories_dir = Path(memories_dir)

    def save_to_memories(self, domain: str, group_key: str, knowledge: Dict):
        """保存知识到JSON"""
        domain_dir = self.memories_dir / domain
        domain_dir.mkdir(parents=True, exist_ok=True)
        json_file = domain_dir / f"{group_key}.json"
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(knowledge, f, ensure_ascii=False, indent=2)
        print(f"  → JSON: {json_file.name}")

    def process_all(self, limit: int = None):
        """处理所有知识文件"""
        print("\n" + "="*80)
        print("🚀 开始完整Pipeline")
        print("="*80 + "\n")

        organized = self.organizer.scan_and_organize()

        for domain, groups in organized.items():
            print(f"\n📂 领域: {domain}")
            print("-" * 80)

            count = 0
            for group_key, group in groups.items():
                if limit and count >= limit:
                    print(f"\n⏸️  达到限制({limit}),停止")
                    break

                print(f"\n[{count+1}] {group.topic}")

                # 加载
                docs = self.loader.load_and_clean(group.primary_file.path)
                if not docs:
                    print("  ⚠️ 加载失败")
                    continue
                print(f"  → 加载: {len(docs)} 页")

                # 提取
                knowledge = self.extractor.extract_from_documents(docs, group.topic)
                self.save_to_memories(domain, group_key, knowledge)

                # 向量化
                self.vector_manager.add_documents(domain, docs,
                    {"domain": domain, "topic": group.topic, "seq": group.sequence})

                count += 1

            print(f"\n✅ {domain}: 完成 {count} 个")

        print("\n" + "="*80)
        print("🎉 Pipeline完成!")
        print("="*80)
        print(f"\n📁 输出:")
        print(f"  - JSON: {self.memories_dir}")
        print(f"  - 向量库: {self.vector_manager.persist_directory}")

print("✅ 步骤5完成")


In [ ]:
# 初始化
processor = KnowledgeProcessor(
    knowledge_base_dir="../knowledge_base",
    memories_dir="./memories/knowledge",
    vector_db_dir="./vector_db"
)

print("✅ Processor初始化完成")


In [ ]:
# 测试: 处理前2个知识块
processor.process_all(limit=2)


In [ ]:
# 正式运行: 处理所有知识块(取消注释)
# processor.process_all()


## 检查输出


In [ ]:
# 查看生成的JSON
memories_path = Path("./memories/knowledge")
if memories_path.exists():
    for domain_dir in memories_path.iterdir():
        if domain_dir.is_dir():
            print(f"\n📂 {domain_dir.name}:")
            for json_file in sorted(domain_dir.glob("*.json")):
                print(f"  - {json_file.name}")
else:
    print("⚠️ 尚未生成输出")


In [ ]:
# 读取第一个JSON查看
json_files = list(Path("./memories/knowledge").rglob("*.json"))
if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample = json.load(f)
    print(f"示例 ({json_files[0].name}):")
    print(json.dumps(sample, ensure_ascii=False, indent=2))
else:
    print("⚠️ 无JSON")


## 总结

### ✅ 功能完整

- 步骤1: 文件扫描分组 ✓
- 步骤2: 文档加载清洗 ✓ (仅PDF)
- 步骤3: LLM结构化提取 ✓
- 步骤4: 向量化存储 ✓
- 步骤5: Pipeline协调 ✓

### 📦 输出

- `memories/knowledge/{domain}/*.json` - 结构化知识
- `vector_db/{domain}/` - 向量数据库

### 🚀 使用

1. 先用`limit=2`测试
2. 确认后`process_all()`处理全部
3. 检查输出结果

### ⚙️ 说明

- **仅支持PDF**: 简化版只处理PDF(主要格式)
- **需API Key**: DeepSeek API (在.env配置)
- **首次下载**: embedding模型会自动下载
